<a href="https://colab.research.google.com/github/Zelechos/Generador-Css/blob/main/GeneradorCSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Generador de CSS [v0.0.7]**

# Clonamos nuestro repositorio en donde se encuentra nuestro dataset con este comando

In [ ]:
!git clone https://github.com/Zelechos/Generador-Css.git

Cloning into 'Generador-Css'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 49 (delta 15), reused 13 (delta 3), pack-reused 0
Unpacking objects: 100% (49/49), done.


### en este caso no es necesario ya que tenemos el dataset en github por ende tenemos que trabajar con el dataset que esta en formato json

### Accedemos a los datos del dataset TEST.json


In [50]:
import json

with open('/content/Generador-Css/dataset/TEST.json') as file:
    data = json.load(file)

#accedemos a la los datos
data_styles = data["styles"]
print(data_styles[0])


#accediendo a los estilos
styles = data_styles[0]["style"]
print("\nstyles access => ",styles)


#accediendo a los selectores
selector = data_styles[0]["selector"]
print("\nselector access => ",selector)


{'style': 'body{overflow:hidden;background:rgb(25,35,125)}', 'selector': 'body'}

styles access =>  body{overflow:hidden;background:rgb(25,35,125)}

selector access =>  body


### Creamos un array para poder mandar los datos y luego tranformarlos en numeros

In [51]:
data_css = []
styles = []
selectors = []
for data in data_styles:
  selectors.append(data["selector"])
  styles.append(data["style"])
 
# print(selectors)
# print(styles)
data_css.append(selectors)
data_css.append(styles)
print(data_css[0])


['body', 'div.drop-container', 'div.drop', 'h1', '@font-face', 'body', '.logo', '.switch-left', '.pad-left', '.switch-right', '.pad-right', '.line', '.text', '.text', '.text', '.reload', '.logo', 'html,body', 'body', '*', '*', '*', '.css_lab', '.witch', 'body', 'body img', 'img', 'body h1', 'h1', 'body a', 'a', '@keyframes glow', 'from', 'to', 'body', 'h1', 'ul', 'li', 'li:hover span', 'li.invisible', 'li.animate', 'span', 'span:before', 'html', 'body', 'header', 'article', 'h3,section', 'h3', 'section', 'h3:before', 'h3:before,section:before', 'section:before', 'h3', 'section', 'h3:before', 'section:before', 'p', 'a', 'body,html', 'body', 'html', 'div.codepen', 'div', '.codepen', '*', 'body', 'aside.context', '.context', 'aside', 'a', 'a:hover', 'footer', 'footer a', 'a']


### Convertimos los datos a numeros para trabajar

### Pasos a Seguir para construir mi Custom Dataset
1.   Contruir el vocabulario para asignar a cada palabra
2.   Configurar un dataset de Pytorch para cargar nuestro datos
2.   Configurar el batch para que todos los ejemplos tengan las misma longitud (seq_len) y tambien configuramos el dataloader




## Importaciones requeridas

In [52]:
import torch
import torch.nn
import spacy # para la tokenizacion
import pandas as pd # para buscar en el archivo de anotaciones (atento)
from torch.nn.utils.rnn import pad_sequence #para el lote de almohadillas (pad batch) para que todos los datos tengan la misma longitud
from torch.utils.data import DataLoader , Dataset #para crea el dataset y el dataloader para el entrenamiento

### Clase para crear nuestro vocabulario

In [55]:
spacy_eng = spacy.load("en")
class Vocabulary:

  def __init__(self, freq_threshold):
    self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}
    self.stoi = {"<PAD>":0, "<SOS>":1, "<EOS>":2, "<UNK>":3}
    self.freq_threshold = freq_threshold

  def __len__(self):
    #devolvemos la longitud de nuestro vocabulario
    return len(self.itos)

  #metodo para definir el ingles de nuestro vocabulario
  @staticmethod
  def tokenizer_eng(text):
    return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

  #ESTO ES LO QUE HACER EJEMPLO!!!
  # "body{background-color:red;}"" => ["body","{","background","-","color",":","red",";","}"]

  #construimos el vocabulario!!
  def build_vocabulary(self, sentence_list):
    frequencies = {}
    idx = 4

    for sentence in sentence_list:
      for word in self.tokenizer_eng(sentence):
        if word not in frequencies:
          frequencies[word] = 1
        else:
          frequencies[word] += 1

        if frequencies[word] == self.freq_threshold:
          self.stoi[word] = idx
          self.itos[idx] = word
          idx += 1

  def numericalize(self, text):
    tokenized_text = self.tokenizer_eng(text)
    return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
    ]

### Clase CssDataset para clasificar el conjunto de datos nuestro dataset

In [57]:
class CssDataset(Dataset):

  def __init__ (self, css_styles, transform = None, freq_threshold = 5):
    self.df = css_styles
    self.transform = transform

    # vamos a obtener nuestros selectores
    self.selectors = self.df[0]

    # vamos a obtener nuestros styles
    self.styles = self.df[1]
    
    #Incializamos el Vocabulario (IMPORTANTEE!!!)
    self.vocab = Vocabulary(freq_threshold)
    self.vocab.build_vocabulary(self.styles) #Pasamos las Listas a el vocabulario

  def __len__(self):
    return len(self.df[1]) #devolvemos la longitud de nuestros dataset

  def __getitem__(self, index):
    selector = self.selectores[index]
    style = self.styles[index]

    numericalized_style = [self.vocab.stoi["<SOS>"]] 
    numericalized_style += self.vocab.numericalize(style) 
    numericalized_style.append(self.vocab.stoi["<EOS>"]) 

    return selector , torch.tensor(numericalized_style) 


## Clase para cargar los datos

In [59]:
class MyCollate:

  def __init__(self, pad_idx):
    self.pad_idx = pad_idx

  def __call__(self, batch):
    selector = [item[0].unsqueeze(0) for item in batch]
    targets = [item[1] for item in batch]
    targets = pad_sequence(targets, batch_first = False, padding_value = self.pad_idx )

    return selector, targets

  def get_loader(css_styles, transform, batch_size = 32, num_workers = 8, shuffle = True, pin_memory = True):
    dataset = CssDataset(css_styles, transform=transform)
    pad_idx = dataset.vocab.stoi["<PAD>"]

    loader = DataLoader(
        dataset = dataset,
        batch_size = batch_size,
        num_workers = num_workers,
        shuffle= shuffle,
        pin_memory=pin_memory,
        collate_fn=MyCollate(pad_idx=pad_idx),
    )

    return loader

  def main():
    dataloader = get_loader(data_css[1],transform=None)

    for idx , (selectors, styles) in enumerate(dataloader):
      print(selectors.shape)
      print(styles.shape)

if __name__ == "__main__" :
  main()



TypeError: ignored